In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import librosa.display
import tensorflow as tf

In [2]:
metadata=pd.read_csv(r"C:\Users\Rashiqua Munshi\Downloads\ProjectSchool\UrbanSound8K.csv")
metadata.head(10)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
5,100263-2-0-143.wav,100263,71.500000,75.500000,1,5,2,children_playing
6,100263-2-0-161.wav,100263,80.500000,84.500000,1,5,2,children_playing
7,100263-2-0-3.wav,100263,1.500000,5.500000,1,5,2,children_playing
8,100263-2-0-36.wav,100263,18.000000,22.000000,1,5,2,children_playing
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn


In [3]:
audio_dataset_path=r"C:\Users\Rashiqua Munshi\Downloads\ProjectSchool\archive"
audio_dataset_path


'C:\\Users\\Rashiqua Munshi\\Downloads\\ProjectSchool\\archive'

In [4]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file) 
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    return mfccs_scaled_features 

In [5]:
import numpy as np
from tqdm import tqdm
import os
### Now we iterate through every audio file and extract features 
### using Mel-Frequency Cepstral Coefficients
extracted_features=[]
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'\\',str(row["slice_file_name"]))
    final_class_labels=row["class"]
    data=features_extractor(file_name)
    extracted_features.append([data,final_class_labels])

3550it [00:48, 73.10it/s] C:\Users\Rashiqua Munshi\Anaconda3\envs\tensorflow1\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8325it [01:51, 97.67it/s] C:\Users\Rashiqua Munshi\Anaconda3\envs\tensorflow1\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
C:\Users\Rashiqua Munshi\Anaconda3\envs\tensorflow1\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [01:56, 74.90it/s] 


In [6]:
### converting extracted_features to Pandas dataframe
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-211.93698, 62.581203, -122.81315, -60.745293...",dog_bark
1,"[-417.0052, 99.336624, -42.995586, 51.073326, ...",children_playing
2,"[-452.39316, 112.36253, -37.578068, 43.195866,...",children_playing
3,"[-406.47922, 91.1966, -25.043556, 42.78452, 11...",children_playing
4,"[-439.63873, 103.86224, -42.658787, 50.690277,...",children_playing


In [7]:
x=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [8]:
x.shape

(8732, 40)

In [9]:
y.shape

(8732,)

In [10]:
x=x.reshape((x.shape[0],1,x.shape[1]))
# x.shape

In [11]:
y

array(['dog_bark', 'children_playing', 'children_playing', ...,
       'car_horn', 'car_horn', 'car_horn'], dtype='<U16')

In [12]:
### Label Encoding
y=np.array(pd.get_dummies(y))
y

array([[False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       ...,
       [False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False]])

In [13]:
### Train Test Split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)

In [14]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.15, random_state=42)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [16]:
### No of classes
num_labels=y.shape[1]

In [17]:
# build network topology
model = keras.Sequential()

# 2 LSTM layers
model.add(keras.layers.LSTM(100, input_shape=(x_train.shape[1],x_train.shape[2]),return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(100))
model.add(keras.layers.Dropout(0.3))

# dense layer
# model.add(keras.layers.Dense(64, activation='relu'))


# output layer
model.add(keras.layers.Dense(10,activation='softmax'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 100)            56400     
                                                                 
 dropout (Dropout)           (None, 1, 100)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 10)                1010      
                                                                 
Total params: 137,810
Trainable params: 137,810
Non-trainable params: 0
_________________________________________________________________


In [19]:

model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [20]:
x_train.shape

(5937, 1, 40)

In [21]:
x_test.shape

(1747, 1, 40)

In [22]:
y_train.shape

(5937, 10)

In [23]:
y_test.shape

(1747, 10)

In [24]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 200
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_valid, y_valid), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/200
182/186 [============================>.] - ETA: 0s - loss: 1.8498 - accuracy: 0.3723
Epoch 1: val_loss improved from inf to 1.37658, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 4s 7ms/step - loss: 1.8438 - accuracy: 0.3749 - val_loss: 1.3766 - val_accuracy: 0.5553
Epoch 2/200
184/186 [============================>.] - ETA: 0s - loss: 1.2539 - accuracy: 0.5805
Epoch 2: val_loss improved from 1.37658 to 1.09690, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 1s 3ms/step - loss: 1.2516 - accuracy: 0.5816 - val_loss: 1.0969 - val_accuracy: 0.6450
Epoch 3/200
184/186 [============================>.] - ETA: 0s - loss: 1.0513 - accuracy: 0.6489
Epoch 3: val_loss improved from 1.09690 to 0.94259, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 1s 3ms/step - loss: 1.0497 - accuracy: 0.6497 - val_loss: 0.9426 - val_accuracy: 0.6918

Epoch 27/200
177/186 [===========================>..] - ETA: 0s - loss: 0.3683 - accuracy: 0.8743
Epoch 27: val_loss improved from 0.50962 to 0.50799, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 1s 4ms/step - loss: 0.3681 - accuracy: 0.8745 - val_loss: 0.5080 - val_accuracy: 0.8397
Epoch 28/200
171/186 [==========================>...] - ETA: 0s - loss: 0.3542 - accuracy: 0.8783
Epoch 28: val_loss did not improve from 0.50799
186/186 [==============================] - 1s 3ms/step - loss: 0.3535 - accuracy: 0.8796 - val_loss: 0.5147 - val_accuracy: 0.8378
Epoch 29/200
170/186 [==========================>...] - ETA: 0s - loss: 0.3383 - accuracy: 0.8873
Epoch 29: val_loss improved from 0.50799 to 0.48797, saving model to saved_models\audio_classification.hdf5
186/186 [==============================] - 1s 4ms/step - loss: 0.3402 - accuracy: 0.8846 - val_loss: 0.4880 - val_accuracy: 0.8435
Epoch 30/200
179/186 [===========================

186/186 [==============================] - 1s 3ms/step - loss: 0.2318 - accuracy: 0.9203 - val_loss: 0.4687 - val_accuracy: 0.8683
Epoch 56/200
170/186 [==========================>...] - ETA: 0s - loss: 0.2123 - accuracy: 0.9309
Epoch 56: val_loss did not improve from 0.45911
186/186 [==============================] - 1s 3ms/step - loss: 0.2134 - accuracy: 0.9293 - val_loss: 0.4965 - val_accuracy: 0.8531
Epoch 57/200
170/186 [==========================>...] - ETA: 0s - loss: 0.2308 - accuracy: 0.9180
Epoch 57: val_loss did not improve from 0.45911
186/186 [==============================] - 1s 3ms/step - loss: 0.2286 - accuracy: 0.9193 - val_loss: 0.4772 - val_accuracy: 0.8683
Epoch 58/200
185/186 [============================>.] - ETA: 0s - loss: 0.2180 - accuracy: 0.9280
Epoch 58: val_loss did not improve from 0.45911
186/186 [==============================] - 1s 3ms/step - loss: 0.2187 - accuracy: 0.9276 - val_loss: 0.4899 - val_accuracy: 0.8540
Epoch 59/200
170/186 [================

Epoch 85/200
171/186 [==========================>...] - ETA: 0s - loss: 0.1678 - accuracy: 0.9393
Epoch 85: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 3ms/step - loss: 0.1654 - accuracy: 0.9412 - val_loss: 0.4922 - val_accuracy: 0.8664
Epoch 86/200
184/186 [============================>.] - ETA: 0s - loss: 0.1714 - accuracy: 0.9448
Epoch 86: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 3ms/step - loss: 0.1712 - accuracy: 0.9448 - val_loss: 0.4916 - val_accuracy: 0.8616
Epoch 87/200
186/186 [==============================] - ETA: 0s - loss: 0.1706 - accuracy: 0.9389
Epoch 87: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 3ms/step - loss: 0.1706 - accuracy: 0.9389 - val_loss: 0.4989 - val_accuracy: 0.8655
Epoch 88/200
171/186 [==========================>...] - ETA: 0s - loss: 0.1532 - accuracy: 0.9483
Epoch 88: val_loss did not improve from 0.45156
186/186 [==============

186/186 [==============================] - 1s 3ms/step - loss: 0.1347 - accuracy: 0.9532 - val_loss: 0.5119 - val_accuracy: 0.8702
Epoch 115/200
172/186 [==========================>...] - ETA: 0s - loss: 0.1384 - accuracy: 0.9519
Epoch 115: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 3ms/step - loss: 0.1411 - accuracy: 0.9510 - val_loss: 0.4614 - val_accuracy: 0.8836
Epoch 116/200
172/186 [==========================>...] - ETA: 0s - loss: 0.1529 - accuracy: 0.9484
Epoch 116: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 3ms/step - loss: 0.1559 - accuracy: 0.9476 - val_loss: 0.4775 - val_accuracy: 0.8721
Epoch 117/200
174/186 [===========================>..] - ETA: 0s - loss: 0.1435 - accuracy: 0.9511
Epoch 117: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 3ms/step - loss: 0.1444 - accuracy: 0.9512 - val_loss: 0.4841 - val_accuracy: 0.8750
Epoch 118/200
172/186 [=========

Epoch 144/200
178/186 [===========================>..] - ETA: 0s - loss: 0.1091 - accuracy: 0.9616
Epoch 144: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 4ms/step - loss: 0.1115 - accuracy: 0.9601 - val_loss: 0.5277 - val_accuracy: 0.8683
Epoch 145/200
177/186 [===========================>..] - ETA: 0s - loss: 0.1121 - accuracy: 0.9617
Epoch 145: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 4ms/step - loss: 0.1122 - accuracy: 0.9618 - val_loss: 0.4754 - val_accuracy: 0.8760
Epoch 146/200
178/186 [===========================>..] - ETA: 0s - loss: 0.1151 - accuracy: 0.9594
Epoch 146: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 4ms/step - loss: 0.1174 - accuracy: 0.9594 - val_loss: 0.5123 - val_accuracy: 0.8645
Epoch 147/200
172/186 [==========================>...] - ETA: 0s - loss: 0.1186 - accuracy: 0.9598
Epoch 147: val_loss did not improve from 0.45156
186/186 [======

186/186 [==============================] - 1s 4ms/step - loss: 0.1035 - accuracy: 0.9640 - val_loss: 0.5144 - val_accuracy: 0.8731
Epoch 174/200
173/186 [==========================>...] - ETA: 0s - loss: 0.1112 - accuracy: 0.9599
Epoch 174: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 4ms/step - loss: 0.1129 - accuracy: 0.9592 - val_loss: 0.5315 - val_accuracy: 0.8693
Epoch 175/200
184/186 [============================>.] - ETA: 0s - loss: 0.1336 - accuracy: 0.9543
Epoch 175: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 3ms/step - loss: 0.1328 - accuracy: 0.9545 - val_loss: 0.5301 - val_accuracy: 0.8674
Epoch 176/200
175/186 [===========================>..] - ETA: 0s - loss: 0.0983 - accuracy: 0.9666
Epoch 176: val_loss did not improve from 0.45156
186/186 [==============================] - 1s 4ms/step - loss: 0.0997 - accuracy: 0.9660 - val_loss: 0.4754 - val_accuracy: 0.8855
Epoch 177/200
185/186 [=========

In [26]:
train_accuracy=model.evaluate(x_train,y_train,verbose=0)
print(train_accuracy[1])

0.9954522252082825


In [25]:
test_accuracy=model.evaluate(x_test,y_test,verbose=0)
print(test_accuracy[1])

0.9055523872375488


In [27]:
!pip install flask

In [28]:
model.save("audio_model")